In [1]:
#For Colab ENV
from google.colab import drive 
drive.mount('/gdrive')
!git clone https://github.com/FacerAin/Kor_Eng_NMT_System.git '/gdrive/My Drive/Colab Notebooks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
Cloning into '/gdrive/My Drive/Colab Notebooks'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 2), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


## NMT_seq2seq_basic

Character Level NMT based on Simple Seq2Seq Model Without Attention Mechanism 

Refer to https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

In [222]:
import numpy as np 
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [223]:
# Parameter Setting
dataset_dir_path = '../dataset/kor-eng/'
batch_size = 64
latent_dim = 256
epochs = 50
num_samples = 2500

In [224]:
# Text data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [225]:
# Read Dataset and Preprocessing
with open(dataset_dir_path+"kor.txt", "r", encoding='utf-8') as f:
    lines = f.read().split('\n')
    for line in lines[:min(num_samples, len(lines)-2)]:#To Except Last Line
        split_line = line.split('\t')
        input_text = split_line[0]
        target_text = '\t' + split_line[1]+ '\n' #Using '^' to start tag, '&' to end tag
        input_texts.append(input_text)
        target_texts.append(target_text)
        for char_item in input_text:
            input_characters.add(char_item)
        for char_item in target_text:
            target_characters.add(char_item)

In [226]:
input_characters = sorted(input_characters)
target_characters = sorted(target_characters)

encoder_token_num = len(input_characters)
target_token_num = len(target_characters)

max_encoder_seq_length = max([len(item) for item in input_texts])
max_decoder_seq_length = max([len(item) for item in target_texts])

encoder_token_index_dict = dict([(char_item, i) for i, char_item in enumerate(input_characters)])
target_token_index_dict = dict([(char_item, i) for i, char_item in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, encoder_token_num), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, target_token_num), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, target_token_num), dtype='float32')

In [227]:
for i,(input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    for j, char_item in enumerate(input_text):
        encoder_input_data[i, j, encoder_token_index_dict[char_item]] = 1.
    encoder_input_data[i, j+1:, encoder_token_index_dict[" "]] = 1.
    for j, char_item in enumerate(target_text):
        decoder_input_data[i, j, target_token_index_dict[char_item]] = 1.
        if j > 0:
            decoder_target_data[i , j-1, target_token_index_dict[char_item]] = 1.
    decoder_input_data[i, j+1: ,target_token_index_dict[" "]] = 1.
    decoder_target_data[i, j:, target_token_index_dict[" "]] = 1.

In [228]:
# Define the Encoder
encoder_inputs = Input(shape = (None, encoder_token_num))
encoder_lstm = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [229]:
#Define the Decoder
decoder_inputs = Input(shape = (None, target_token_num))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(target_token_num, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [230]:
#Define the Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [231]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('s2s.h5')   

Epoch 1/50
32/32 [==============================] - 23s 721ms/step - loss: 2.3319 - accuracy: 0.6633 - val_loss: 2.6010 - val_accuracy: 0.5753
Epoch 2/50
32/32 [==============================] - 21s 658ms/step - loss: 1.8088 - accuracy: 0.6896 - val_loss: 2.8063 - val_accuracy: 0.5753
Epoch 3/50
32/32 [==============================] - 22s 681ms/step - loss: 1.7642 - accuracy: 0.6904 - val_loss: 2.4876 - val_accuracy: 0.5753
Epoch 4/50
32/32 [==============================] - 21s 665ms/step - loss: 1.7263 - accuracy: 0.6908 - val_loss: 2.5807 - val_accuracy: 0.5753
Epoch 5/50
32/32 [==============================] - 20s 614ms/step - loss: 1.6684 - accuracy: 0.6982 - val_loss: 2.6286 - val_accuracy: 0.5782
Epoch 6/50
32/32 [==============================] - 20s 610ms/step - loss: 1.5959 - accuracy: 0.7193 - val_loss: 2.5011 - val_accuracy: 0.6016
Epoch 7/50
32/32 [==============================] - 19s 600ms/step - loss: 1.5332 - accuracy: 0.7338 - val_loss: 2.2900 - val_accuracy: 0.6118

In [232]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [233]:
reverse_input_char_index_dict = dict(
    (i, char) for char, i in encoder_token_index_dict.items())
reverse_target_char_index_dict = dict(
    (i, char) for char, i in target_token_index_dict.items())

In [234]:
def decode_seq(input_seq):
    states_value =  encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, target_token_num))
    target_seq[0,0,target_token_index_dict['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq]+states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index_dict[sampled_token_index]
        decoded_sentence += sampled_char

        if(sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1,1,target_token_num))
        target_seq[0,0,sampled_token_index] = 1.

        states_value = [h,c]
    return decoded_sentence

In [235]:
print(input_texts[2000])
print(encoder_input_data[2000][3])
print(decode_seq(encoder_input_data[2000:2001]))

Somebody was talking to Tom.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
그 사람은 아직 도도 했어.



In [236]:
for idx in range(200):
    print(idx)
    input_seq = encoder_input_data[idx: idx+1]
    decoded_sent = decode_seq(input_seq)
    print('Input:', input_texts[idx])
    print('output:', decoded_sent)


0
Input: Go.
output: 그만해.

1
Input: Hi.
output: 이거 가져.

2
Input: Run!
output: 빨리!

3
Input: Run.
output: 그만해.

4
Input: Who?
output: 이거 가져.

5
Input: Wow!
output: 그만해!

6
Input: Fire!
output: 빨리!

7
Input: Help!
output: 빨리!

8
Input: Jump!
output: 빨리!

9
Input: Jump.
output: 그만해.

10
Input: Wait!
output: 그만 !

11
Input: Wait!
output: 그만 !

12
Input: Wait.
output: 그만해.

13
Input: Begin.
output: 그만해.

14
Input: Hello!
output: 빨리!

15
Input: I see.
output: 이건 내가 수가 있어.

16
Input: I try.
output: 이건 내가 수가 있어.

17
Input: I won!
output: 난 톰이 아직도 말해.

18
Input: Oh no!
output: 그만해.

19
Input: Relax.
output: 그만해.

20
Input: Shoot!
output: 빨리!

21
Input: Smile.
output: 그만 사람들이 아.

22
Input: Attack!
output: 빨리!

23
Input: Attack!
output: 빨리!

24
Input: Freeze!
output: 빨리!

25
Input: Get up.
output: 그만해.

26
Input: Got it!
output: 빨리!

27
Input: Hug me.
output: 그만해.

28
Input: I know.
output: 난 톰이 아직도 말 했어.

29
Input: I work.
output: 난 톰을 아직도 말 했어.

30
Input: Listen.
output: 그만 사람들은 아주 줘.

31
Input